In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widget
from IPython.display import display
import scipy.signal as sig
%matplotlib inline

pi = np.math.pi

def setLabels(lX, lY):
    plt.xlabel(lX)
    plt.ylabel(lY)

plt.rcParams['font.size'] = 40
plt.rcParams['figure.figsize'] = (40, 10)
plt.rcParams['lines.linewidth'] = 2.0
plt.rcParams['figure.subplot.hspace'] = 0.4

# Aufgabe 2: Faltung und Transformationstheoreme

Die Impulsantwort eines LTI-Systems lautet:
\begin{align}
    h(t) = \mathrm{rect}\left(\frac{t}{T}\right), \quad T > 0.
\end{align}

#### Graphische Darstellung der Impulsantwort

In [ ]:
N = 3*1000
T = 1
T_d = 6*T
samples_per_unit = N/T_d
crest_factor = samples_per_unit

t = np.linspace(-T_d/2, T_d/2, N)
h = np.zeros(t.shape)
h[(t >= -T/2) & (t <= T/2)] = 1
plt.grid()
plt.xlim((-3, 3))
plt.ylim((-0.1, 1.1))
plt.axvline(linewidth=0.75, color='k')
plt.plot(t/T, h, 'b-')
plt.axhline(linewidth=0.75, color='k')
setLabels(r'$t/T$', r'$h(t)$')

## 2.1
Ist das System kausal?

Für ein kausales System muss gelten:
\begin{align}
    h(t) = 0 \quad \forall t < 0
\end{align}
Das System ist nicht kausal, da $h(t) \neq 0$ für $t \in [-T/2, 0] < 0$ gilt.

## 2.2
Bestimmen **und** skizzieren Sie die Antwort $y_1(t)$ des Systems auf das Eingangssignal $x_1(t) = \mathrm{rect}\left(\frac{t}{T}\right)$.

\begin{align}
        y_1(t) &= x_1(t) \ast h(t) \\
    &= \mathrm{rect}\left( \frac{t}{T} \right) \ast \mathrm{rect}\left( \frac{t}{T} \right)
    \end{align}
#### Faltungssatz:
\begin{align}
    \mathcal{F}\left\{ x_1(t) \ast x_2(t) \right\} = X_1(j\omega) \cdot X_2(j\omega)
    \end{align}
#### Ähnlichkeitssatz:
\begin{align}
    &\mathcal{F}\left\{x(at) \right\} = \frac{1}{|a|} X \left(j\frac{\omega}{a} \right) \\
    \Rightarrow &\mathcal{F}\left\{ \mathrm{rect}\left(\frac{t}{T}\right) \right \} = T \mathrm{si}\left( \frac{\omega T}{2} \right)
\end{align}
#### a) Lösung im Frequenzbereich:
\begin{align}
    Y_1(j\omega) &= X_1(j\omega) \cdot H(j\omega) \\
    &= \left( T \mathrm{si}\left( \frac{\omega T}{2} \right) \right) \cdot \left( T \mathrm{si}\left( \frac{\omega T}{2} \right) \right) \\
    &= T \cdot T\mathrm{si}^2\left( \frac{\omega T}{2} \right) \\
    \Rightarrow y_1(t) &= T \cdot \mathcal{F}^{-1}\left\{ T\mathrm{si}^2\left( \frac{\omega T}{2} \right) \right\} \\
    &= T \mathrm{\Lambda} \left(\frac{t}{T}\right)
\end{align}
#### b) (graphische Lösung)

#### Graphische Darstellung von $y_1(t)$

In [ ]:
# Define x_1(t) here:
x1 = h
#x1 = np.copy(np.roll(h, -int(samples_per_sec)))
#omega_s = 2*pi*(1/(T/8))
#x1 = np.sin(omega_s*t);

# define h(t) here:
h_local = h 
#tau = T_d/12
#h_local = (t >= 0)*1/tau*np.exp(-t/tau)

# get y_2(t)
y1 = sig.convolve(h_local, x1, 'same')/crest_factor

zeroFunc = np.zeros(h.shape)
ax1_ylower = min(min(x1), min(h))
ax1_yupper = max(max(x1), max(h))

ax2_ylower = min(y1)
ax2_yupper = max(y1)
def LTI_convolution(shift):
    shift_in_samples = int(shift * samples_per_unit)    

    h_shifted_and_inverted = np.roll(np.flipud(h_local), shift_in_samples)
    temp = x1 * h_shifted_and_inverted;
    y1_current_value = np.trapz(temp)/crest_factor

    # first plot: impulse response and input signal
    # area under product
    fig1 = plt.figure();
    ax1 = fig1.add_subplot(2, 1, 1,
        xlim = (-1.5*T, 1.5*T),
        ylim = (1.1*ax1_ylower, 1.1*ax1_yupper),
        xlabel = r'$\tau/T$',
        ylabel = r'$x_1(\tau), h(-(\tau-t))$')

    ax1.axvline(linewidth=0.75, color='k')
    ax1.plot(t/T, x1, color = 'k')
        
    ax1.plot(t/T, h_shifted_and_inverted, color='b')
    ax1.fill_between(t/T, 0, temp, facecolor='r')
    ax1.axhline(linewidth=0.75, color='k')
    ax1.grid()

    # second plot: output signal
    ax2 = fig1.add_subplot(2, 1, 2,
        xlim = (-1.5*T, 1.5*T),
        ylim = (1.1*ax2_ylower, 1.1*ax2_yupper),
        xlabel = r'$t/T$',
        ylabel = r'$y_1(t)$')
    ax2.axvline(linewidth=0.75, color='k')
    ax2.plot(t/T, y1, 'b-')
    ax2.grid()
    mline, sline, baseline = ax2.stem([shift],
            [y1_current_value], color='r')
    plt.setp(mline, 'markerfacecolor', 'r')
    plt.setp(sline, color='r')
    ax2.axhline(linewidth=0.75, color='k')
    

wdg1 = widget.interactive(LTI_convolution, shift = 
    widget.FloatSlider(min = -T, max = T, step = 0.1, value = 0.0))
ret = display(wdg1)

## 2.3
Bestimmen **und** skizzieren sie die Antwort $y_2(t)$ des Systems auf das Eingangssignal $x_2(t) = \mathrm{rect}\left(\frac{2 t}{T} - \frac{3}{2}\right)$.

#### Umschreiben des Signals:
\begin{align}
    x_2(t) &= \mathrm{rect} \left( \frac{2 t}{T} - \frac{3}{2} \right) \\
    &= \mathrm{rect} \left( \frac{ 2 t - 3T/2}{T} \right) \\
    &= \mathrm{rect} \left( \frac{ t - 3T/4}{T/2} \right) \\
    &= \underbrace{ \mathrm{rect} \left( \frac{t}{T/2} \right)}_{:= \tilde{x}_2(t)} \ast \delta\left(t - 3T/4\right)
\end{align}

#### Faltung mit unverschobenen Signalen:
\begin{align}
    \tilde{y}_2(t) &= \tilde{x}_2(t) \ast h(t) \\
        &= \mathrm{rect}\left( \frac{t}{T/2} \right) \ast \mathrm{rect}\left( \frac{t}{T} \right)
\end{align}

#### a) Lösung mit Zerlegung:
\begin{align}
    \mathrm{rect}\left( \frac{t}{T} \right) &= \mathrm{rect}\left( \frac{t-t/4}{T/2} \right) + \mathrm{rect}\left( \frac{t+T/4}{T/2} \right) \\
    &= \mathrm{rect}\left( \frac{t}{T/2} \right) \ast \left( \delta\left(t - T/4\right) + \delta\left(t + T/4\right)  \right) \\[1em]
    \Rightarrow \tilde{y}_2(t) &= \mathrm{rect}\left( \frac{t}{T/2} \right) \ast \mathrm{rect}\left( \frac{t}{T/2} \right) \ast \left( \delta\left(t - T/4\right) + \delta\left(t + T/4\right) \right) \\
    &\underset{\text{äquiv. zu 2.1)}}{=} \frac{T}{2} \left( \mathrm{\Lambda}\left(\frac{t-T/4}{T/2}\right) + \mathrm{\Lambda}\left(\frac{t+T/4}{T/2}\right) \right) \\[1em]
    \Rightarrow y_2(t) &= \tilde{y}_2(t) \ast \delta(t - 3T/4) = \frac{T}{2} \left( \mathrm{\Lambda}\left(\frac{t-T}{T/2}\right) + \mathrm{\Lambda}\left(\frac{t-T/2}{T/2}\right) \right)
\end{align}

#### Graphische Darstellung von $y_2(t)$:

In [ ]:
r = np.zeros(t.shape)
r[(t >= -T/4) & (t <= T/4)] = 1
tri = sig.convolve(r, r, 'same')/(crest_factor)

u1 = np.roll(tri, int(T/2*crest_factor))
u2 = np.roll(tri, int(T*crest_factor))

plt.axvline(linewidth=0.75, color='k')
plt.plot(t/T, u1, 'r-')
plt.plot(t/T, u2, 'r-')
plt.plot(t/T, u1 + u2, 'b-')
plt.axhline(linewidth=0.75, color='k')
plt.grid()
plt.xlim((-T/2, 2*T))
plt.ylim((-0.1, 1.1*max(u1)))
setLabels(r'$t/T$', r'$y_2(t)$')

#### b) Teilweises Lösen im Frequenzbereich:
\begin{align}
    \mathcal{F}\left\{ \tilde{y}_2(t) \right\} &= \mathcal{F} \left\{ \mathrm{rect}\left( \frac{t}{T/2} \right) \ast \mathrm{rect}\left( \frac{t}{T} \right) \right\} \\
    &= \mathcal{F}\left \{ \mathrm{rect}\left( \frac{t}{T/2} \right) \right\} \cdot \mathcal{F}\left \{ \mathrm{rect}\left( \frac{t}{T} \right) \right\} \\
        &= T \mathrm{si} \left( \frac{\omega T}{2} \right) \cdot \frac{T}{2} \mathrm{si} \left( \frac{\omega T}{4} \right) \\
\end{align}

Hilfsrechnung:
\begin{align}
    \mathrm{si}\left( \frac{\omega T}{2} \right) = \frac{ \mathrm{e}^{j \omega \frac{T}{2}} - \mathrm{e}^{-j \omega \frac{T}{2}} }{2j \frac{\omega T}{2} }
    \underset{\text{3. Bin. Formel}}{=} \frac{ \mathrm{e}^{j \omega \frac{T}{4}} - \mathrm{e}^{-j \omega \frac{T}{4}} }{2j \frac{\omega T}{4} } \cdot \frac{ \mathrm{e}^{j \omega \frac{T}{4}} + \mathrm{e}^{-j \omega \frac{T}{4}} }{2} \quad 
    \left( = \mathrm{si}\left( \frac{\omega T}{2} \right) \cos \left( \frac{\omega T}{4} \right) \right)
\end{align}
#### Damit folgt:
\begin{align}
    \Rightarrow \tilde{Y}_2(j\omega) = \mathcal{F}\left\{ \tilde{y}_2(t) \right\} &= \left(\frac{T}{2}\right)^2 \mathrm{si}^2 \left( \frac{\omega T}{4} \right) \cdot \left( \mathrm{e}^{-j \omega \frac{T}{4}} + \mathrm{e}^{j \omega \frac{T}{4}} \right) \\[1em]
    \Rightarrow \tilde{y}_2(t) &= \frac{T}{2}\cdot \mathrm{\Lambda}\left(\frac{t}{T/2}\right) \ast \left(\delta(t-T/4) + \delta(t+t/4) \right) \\
        &= \frac{T}{2} \left( \mathrm{\Lambda}\left(\frac{t-T/4}{T/2}\right) + \mathrm{\Lambda}\left(\frac{t+T/4}{T/2}\right) \right) \\
    \Rightarrow y_2(t) &= \tilde{y}_2(t) \ast \delta(t - 3T/4) = \left( \mathrm{\Lambda}\left(\frac{t-T}{T/2}\right) + \mathrm{\Lambda}\left(\frac{t-T/2}{T/2}\right) \right)    
\end{align}    

## 2.4
Berechnen sie die Fouriertransformierte $Y_1(j\omega)$ des Ausgangssignals $y_1(t)$.

\begin{align}
        Y_1(j\omega) &= \mathcal{F} \left\{y_1(t) \right \} \\
        &= \mathcal{F}\left\{ x_1(t) \ast h(t) \right\} \\
        &= X_1(j\omega) \cdot H(j\omega) \\ 
        \text{(Ähnlichkeitstheorem / 2.1a)} &= T \mathrm{si}\left( \frac{\omega T}{2} \right) \cdot T \mathrm{si}\left( \frac{\omega T}{2} \right)  \\
        &= T^2 \mathrm{si}^2 \left( \frac{\omega T}{2} \right)
\end{align}
oder
\begin{align}
    Y_1(j\omega) &= \mathcal{F} \left\{y_1(t) \right \} \\
    &=  \mathcal{F} \left\{T \mathrm{\Lambda} \left(\frac{t}{T} \right) \right \} \\
    \text{(Tabelle und Ähn.-Theorem)} &= T\cdot T \mathrm{si}^2\left( \frac{\omega T}{2} \right)
\end{align}    

## 2.5
Berechnen sie die Fouriertransformierte $Y_2(j\omega)$ des Ausgangssignals $y_2(t)$.

Aus 2.3):
\begin{align}
    Y_2(j\omega) &= \tilde{Y}_2(j\omega) \cdot \mathrm{e}^{-j \omega \frac{3T}{4}} \\
    &= \left(\frac{T}{2}\right)^2 \mathrm{si}^2 \left( \frac{\omega T}{4} \right) \cdot \left( \mathrm{e}^{-j \omega \frac{T}{4}} + \mathrm{e}^{j \omega \frac{T}{4}} \right) \cdot \mathrm{e}^{-j \omega \frac{3T}{4}} \\
&= \left(\frac{T}{2}\right)^2 \mathrm{si}^2 \left( \frac{\omega T}{4} \right) \cdot \left( \mathrm{e}^{-j \omega T} + \mathrm{e}^{-j \omega \frac{T}{2}} \right)
\end{align}
oder:
\begin{align}
    Y_2(j\omega) &= \mathcal{F}^{-1}\left\{ y_2(t) \right\}  \\
    &= \mathcal{F}^{-1} \left\{ \frac{T}{2} \left( \mathrm{\Lambda}\left( \frac{t-T}{T/2} \right) + \mathrm{\Lambda}\left( \frac{t-T/2}{T/2} \right) \right) \right\} \\
    &= \left(\frac{T}{2}\right)^2 \mathrm{si}^2 \left( \frac{\omega T}{4} \right) \cdot \left( \mathrm{e}^{-j \omega T} + \mathrm{e}^{-j \omega \frac{T}{2}} \right)
\end{align}

## 2.6
Berechnen sie die Fouriertransformierte des Signals $x_3(t) = \mathrm{si}^2 \left(t \right)$. 

#### Dualitätstheorem:
\begin{align}
    \mathcal{F} \left\{ x(t) \right\} = X(j\omega) \Rightarrow \mathcal{F} \left\{ \frac{1}{2\pi} X(t) \right\} = x(-j\omega)
\end{align}
#### Damit gilt:
\begin{align}
    \mathcal{F} \left\{ \mathrm{\Lambda}(t) \right\} &= \mathrm{si}^2 \left( \frac{\omega}{2} \right) \\
    \Rightarrow \mathcal{F} \left\{ \frac{1}{2 \pi} \mathrm{si}^2 \left( \frac{t}{2} \right) \right\} &= \mathrm{\Lambda}(-\omega) = \mathrm{\Lambda}(\omega)
\end{align}
#### Der Ähnlichkeitssatz liefert dann:
\begin{align}
    \mathcal{F} \left\{ \mathrm{si}^2 \left( 2 \frac{t}{2} \right) \right\} = \frac{2\pi}{2} \mathrm{\Lambda} \left( \frac{\omega}{2} \right) = \pi \mathrm{\Lambda} \left( \frac{\omega}{2} \right)
\end{align}

## Zusatz: Wozu dient dieses System?

Um den Zweck des Systems zu erkennen, wird eine normierte Variante betrachtet:
\begin{align}
    g(t) =  \frac{1}{T} \mathrm{rect}\left(\frac{t}{T}\right)
\end{align}  
Damit lässt sich die Faltung zur Berechnung des Ausgangssignals folgendermaßen ausschreiben:  
  
\begin{align}
    y(t) = x(t) \ast g(t) &= \frac{1}{T}\int_{-\infty}^{\infty} x(\tau) \, \mathrm{rect}\left(\frac{t-\tau}{T}\right) \mathrm{d}\tau \\
    &= \frac{1}{T}\int_{-\infty}^{\infty} x(\tau) \, \mathrm{rect}\left(-\frac{\tau-t}{T}\right) \mathrm{d}\tau \\
    &= \frac{1}{T}\int_{t-\frac{T}{2}}^{t+\frac{T}{2}} x(\tau) \ \mathrm{d}\tau
\end{align}
Das System berechnet also den arithmetischen Mittelwert des Eingangssignals über die Dauer $T$.
Das nicht normierte System berechnet dementsprechend das Integral über die Dauer $T$.